In [1]:
from data import data_loader
from all_coefs import all_coefs,load_coefs
from coefs import weight_matrix
from spread import spread_manager
from revert import df_to_inv_mom,df_to_dict
from pnl import pnl
from pnl import pnl as pnl_engine
from scipy.optimize import minimize

import pandas as pd
import datetime as dt
import numpy as np
import tensorflow as tf

import time

In [2]:
data=data_loader('2024-01-01') #data_obj with all precomputes
price_data=data.load_price()

price_dict=df_to_dict(price_data) #Price Dictionary

In [3]:
"""
#CREATE COEF MATRIX
coefs=all_coefs(data)
coefs.run(t_stat=0.25,weight_imb=0.5,corr_thresh=0.85,weight_type='std')
print(coefs.errors())
coefs.coefs()
coefs.dump_matrix('t025imb05corr085typestd.pkl')
"""

"\n#CREATE COEF MATRIX\ncoefs=all_coefs(data)\ncoefs.run(t_stat=0.25,weight_imb=0.5,corr_thresh=0.85,weight_type='std')\nprint(coefs.errors())\ncoefs.coefs()\ncoefs.dump_matrix('t025imb05corr085typestd.pkl')\n"

In [4]:
coef_matrix=load_coefs(data,'t025imb05corr085typeabs.pkl') #Load COEF MATRIX

dates_used=data.all_dates()[data.all_dates()>=pd.Timestamp(dt.datetime(2021,12,31))] #ISOLATE DATES FOR BACKTEST

In [5]:
spreads=spread_manager(price_data,coef_matrix)
spreads.calc_costs()
signals=spreads.raw_signal()

z_score=spreads.new_z_score()
rev=df_to_inv_mom(z_score)

filterer=spreads.signal_z_score_filter()

In [6]:
signal_dict=df_to_dict(signals)
rev_dict=df_to_dict(rev)
filter_dict=df_to_dict(filterer)

In [7]:
#ALL VALUES THAT NEED DATES

date_old=dates_used[0]
date_new=dates_used[1]

weight_data=weight_matrix(price_dict[date_old],coef_matrix)

betas=data.load_data(date_old,dtype='beta')
covs=data.load_data(date_old,dtype='cov')

In [8]:
initial_weights=price_dict[dates_used[0]].copy()
initial_weights.loc[:]=0
initial_weights

Ticker
FOX      0.0
NOW      0.0
AMCR     0.0
JPM      0.0
KO       0.0
        ... 
CCI      0.0
APTV     0.0
LVS      0.0
MOH      0.0
CONST    0.0
Name: 2021-12-31 00:00:00, Length: 498, dtype: float64

In [9]:
initial_value=100e6

#Signal
c_sig=signal_dict[date_old][filter_dict[date_old]]

#Reversion Speed
c_rev=rev_dict[date_old][filter_dict[date_old]]
lam_1=0

#Weight Matrix
c_A=weight_data[filter_dict[date_old]].T

#Initial Weights DIRECT INPUT
c_w0=initial_weights.copy()
c_w0.drop('CONST',inplace=True)
lam_2=0

#COV Matrix
c_cov=data.load_data(date_old,dtype='cov')
lam_3=1

#Betas
c_beta=data.load_data(date_old,dtype='beta')

theta_1=0.2 #Cash Neutrality
theta_2=0.2 #Beta Neutrality
theta_3=3.8 #Lower Bound Target Total Capital Deployed
theta_4=4.2 #Upper Bound Target Total Capital Deployed


In [10]:
#INITIALIZE TRADES (RANDOM INPUT)
t_init=np.random.normal(size=sum(filter_dict[date_old]))
t_init=t_init/(2.5*np.sum(np.abs(t_init)))

In [11]:
t=t_init #TRANSLATION FOR OBJECTIVE FUNCTION

In [12]:
t=4*-np.ones(t.shape)/t.shape[0]

In [13]:
#Term 1 (MAX RETURN)
print(t@c_sig)
#Term 2 (MAX REV)
print(lam_1*(t@c_rev))
#Term 3 (TRANSACTION COSTS)
print(lam_2*sum(abs((c_A@t)-c_w0)))
#Term 4 (Volatility)
print(lam_3*((c_A@t).T@c_cov@(c_A@t)))

-0.10020793787975771
0.0
0.0
0.012645747549458396


In [14]:
t = tf.Variable(np.zeros(c_sig.shape[0]), dtype=np.float64)

In [15]:

def obj_func(t):
    #OPTIMIZATION
    ret = tf.reduce_sum(t * c_sig)
    rev = tf.reduce_sum(t * c_rev)
    c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))
    tcosts = tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
    vol=tf.reduce_sum(tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w))

    #PENALTIES
    calc_cash=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(np.ones(c_beta.shape), axis=1),c_w,transpose_a=True)))
    cash_penalty=(tf.maximum(theta_1,calc_cash)-theta_1)

    calc_beta=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(c_beta, axis=1),c_w,transpose_a=True)))
    beta_penalty=(tf.maximum(theta_2,calc_beta)-theta_2)

    total_weight=tf.reduce_sum(tf.matmul(tf.abs(c_w),np.ones(c_w.shape),transpose_a=True))
    weight_penalty=(tf.maximum(total_weight,theta_4)-theta_4)+(theta_3-tf.minimum(total_weight,theta_3))
    #print(ret,rev,tcosts,vol)
    #print(cash_penalty,beta_penalty,weight_penalty)
    return (-(10*ret + lam_1*rev - lam_2*tcosts - 10*lam_3*vol))+(1e6*(cash_penalty+beta_penalty+weight_penalty))


def obj_func_print(t):
    #OPTIMIZATION
    ret = tf.reduce_sum(t * c_sig)
    rev = tf.reduce_sum(t * c_rev)
    c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))
    tcosts = tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
    vol=tf.reduce_sum(tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w))

    #PENALTIES
    calc_cash=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(np.ones(c_beta.shape), axis=1),c_w,transpose_a=True)))
    cash_penalty=(tf.maximum(theta_1,calc_cash)-theta_1)

    calc_beta=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(c_beta, axis=1),c_w,transpose_a=True)))
    beta_penalty=(tf.maximum(theta_2,calc_beta)-theta_2)

    total_weight=tf.reduce_sum(tf.matmul(tf.abs(c_w),np.ones(c_w.shape),transpose_a=True))
    weight_penalty=(tf.maximum(total_weight,theta_4)-theta_4)+(theta_3-tf.minimum(total_weight,theta_3))
    print(ret,rev,tcosts,vol)
    print(cash_penalty,beta_penalty,weight_penalty)
    return (-(10*ret + lam_1*rev - lam_2*tcosts - lam_3*vol))+(1e6*(cash_penalty+beta_penalty+weight_penalty))

In [16]:
def compute_gradients(t):
    with tf.GradientTape() as tape:
        loss = obj_func(t)
    return tape.gradient(loss, t)

In [17]:
"""
initial_learning_rate = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=50,
    decay_rate=0.8,
    staircase=True
)
optimizer = tf.optimizers.Adam(learning_rate=lr_schedule)

n_epochs = 1000
for epoch in range(n_epochs):
    grads = compute_gradients(t)
    optimizer.apply_gradients(zip([grads], [t]))
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Objective value: {obj_func(t).numpy()}")

# Final optimized parameters
print("Optimized parameters:", t.numpy())
"""

'\ninitial_learning_rate = 1e-5\nlr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(\n    initial_learning_rate,\n    decay_steps=50,\n    decay_rate=0.8,\n    staircase=True\n)\noptimizer = tf.optimizers.Adam(learning_rate=lr_schedule)\n\nn_epochs = 1000\nfor epoch in range(n_epochs):\n    grads = compute_gradients(t)\n    optimizer.apply_gradients(zip([grads], [t]))\n    if epoch % 10 == 0:\n        print(f"Epoch {epoch}, Objective value: {obj_func(t).numpy()}")\n\n# Final optimized parameters\nprint("Optimized parameters:", t.numpy())\n'

In [18]:
obj_func_print(t)


tf.Tensor(0.0, shape=(), dtype=float64) tf.Tensor(0.0, shape=(), dtype=float64) tf.Tensor(0.0, shape=(), dtype=float64) tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(0.0, shape=(), dtype=float64) tf.Tensor(0.0, shape=(), dtype=float64) tf.Tensor(3.8, shape=(), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=3800000.0>

In [19]:
np.sqrt(0.002980875488578272)

0.05459739452188421

In [20]:
#TENSORFLOW EQUIVALENTS
ret = tf.reduce_sum(t * c_sig)
rev = lam_1 * tf.reduce_sum(t * c_rev)
c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))
tcosts = lam_2 *tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
vol=lam_3*tf.reduce_sum(tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w))

In [21]:
#PENALTIES
calc_cash=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(np.ones(c_beta.shape), axis=1),c_w,transpose_a=True)))
cash_penalty=(tf.maximum(theta_1,calc_cash)-theta_1)

calc_beta=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(c_beta, axis=1),c_w,transpose_a=True)))
beta_penalty=(tf.maximum(theta_2,calc_beta)-theta_2)

total_weight=tf.reduce_sum(tf.matmul(tf.abs(c_w),np.ones(c_w.shape),transpose_a=True))
weight_penalty=(tf.maximum(total_weight,theta_4)-theta_4)+(theta_3-tf.minimum(total_weight,theta_3))

In [22]:
def obj_func(t):
    ret=(t@c_sig)
    rev=(lam_1*(t@c_rev))
    tcosts=(lam_2*sum(abs((c_A@t)-c_w0)))
    vol=(lam_3*((c_A@t).T@c_cov@(c_A@t)))
    return -(ret+rev-tcosts-vol)

theta_1=0.2 #Cash Neutrality
theta_2=0.2 #Beta Neutrality
theta_3=0.5 #Max Total Capital to All Trades
theta_4=0.01 #Max Cap per Trade

constraints = (
    {'type': 'ineq', 'fun': lambda t: theta_1 - abs((c_A@t)@np.ones(c_beta.shape))},  #partial cash neutrality
    {'type': 'ineq', 'fun': lambda t: theta_2 - abs((c_A@t)@c_beta)}, #partial beta neutrality
    {'type': 'ineq', 'fun': lambda t: theta_3 - abs(t)@np.ones(t.shape)} #max total cap to trades
)

bounds = [(-theta_4, theta_4)]*c_A.shape[1]

In [23]:
cash_neutrality=abs((c_A@t)@np.ones(c_beta.shape))
beta_neutrality=abs((c_A@t)@c_beta)
print(cash_neutrality,beta_neutrality)

0.0 0.0


In [24]:
c_A.shape

(497, 12752)

In [ ]:
class optimizer:
    def __init__(self,data_obj,signal_dict,rev_dict,price_dict,filter_dict,coef_matrix):
        self.data=data_obj
        self.signal_dict=signal_dict
        self.rev_dict=rev_dict
        self.price_dict=price_dict
        self.filter_dict=filter_dict
        self.pnl=pnl(data_obj)
        self.coef_matrix=coef_matrix

    def optimize(self,c_w0,date_old,
                 lam_1, #Mean Reversion
                 lam_2, #Trade Costs
                 lam_3, #Volatility
                 t=None, #Initial Trades (store as tf Variable)
                 theta_1=0.2, #Cash
                 theta_2=0.2, #Beta
                 theta_3=3.8, #Min Total Inv
                 theta_4=4.2, #Max Total Inv,
                 n_epochs=1000, #Epochs of Optimizer
                 initial_learning_rate=1e-5
                 ):
        #Load all variables

        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=25,
            decay_rate=0.9,
            staircase=True,
        )
        self.optimizer = tf.optimizers.Adam(learning_rate=lr_schedule)

        c_sig=self.signal_dict[date_old][self.filter_dict[date_old]]
        t = tf.Variable(np.zeros(c_sig.shape[0]), dtype=np.float64) if t is None else t
        c_rev=self.rev_dict[date_old][self.filter_dict[date_old]]
        c_A=weight_matrix(self.price_dict[date_old],self.coef_matrix)[filter_dict[date_old]].T
        c_cov=self.data.load_data(date_old,dtype='cov')
        c_beta=self.data.load_data(date_old,dtype='beta')

        def obj_func(t):
            #OPTIMIZATION
            ret = tf.reduce_sum(t * c_sig)
            rev = tf.reduce_sum(t * c_rev)
            c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))

            tcosts = tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
            vol=tf.reduce_sum(tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w))
            #PENALTIES
            calc_cash=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(np.ones(c_beta.shape), axis=1),c_w,transpose_a=True)))
            cash_penalty=(tf.maximum(theta_1,calc_cash)-theta_1)

            calc_beta=tf.abs(tf.reduce_sum(tf.matmul(tf.expand_dims(c_beta, axis=1),c_w,transpose_a=True)))
            beta_penalty=(tf.maximum(theta_2,calc_beta)-theta_2)

            total_weight=tf.reduce_sum(tf.matmul(tf.abs(c_w),np.ones(c_w.shape),transpose_a=True))
            weight_penalty=(tf.maximum(total_weight,theta_4)-theta_4)+(theta_3-tf.minimum(total_weight,theta_3))
            #print(ret,rev,tcosts,vol)
            #print(cash_penalty,beta_penalty,weight_penalty)
            return (-(10*ret + lam_1*rev - lam_2*tcosts - lam_3*vol))+(1e6*(cash_penalty+beta_penalty+weight_penalty))

        def compute_gradients(t):
            with tf.GradientTape() as tape:
                loss = obj_func(t)
            return tape.gradient(loss, t)
        
        for epoch in range(n_epochs):
            grads = compute_gradients(t)
            self.optimizer.apply_gradients(zip([grads], [t]))
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Objective value: {obj_func(t).numpy()}")
        
        return t,c_A@t,c_beta,c_cov
        

In [33]:
opt=optimizer(data,signal_dict,rev_dict,price_dict,filter_dict,coef_matrix)

In [34]:
dates_used=data.all_dates()[data.all_dates()>=pd.Timestamp(dt.date(2021,12,31))]

In [129]:
date_old=dates_used[0]
date_new=dates_used[1]

t,w,beta,cov=opt.optimize(np.array([float(0)]*497).T,date_old,lam_1=0.2,lam_2=0.1,lam_3=20,n_epochs=101)
t,w,beta,cov=opt.optimize(np.array([float(0)]*497).T,date_old,lam_1=0.2,lam_2=0.1,lam_3=20,n_epochs=101,t=t)
t,w,beta,cov=opt.optimize(np.array([float(0)]*497).T,date_old,lam_1=0.2,lam_2=0.1,lam_3=20,n_epochs=101,t=t)
t,w,beta,cov=opt.optimize(np.array([float(0)]*497).T,date_old,lam_1=0.2,lam_2=0.1,lam_3=20,n_epochs=101,t=t)

Epoch 0, Objective value: 3620090.828518039
Epoch 10, Objective value: 908105.5312119789
Epoch 20, Objective value: 596795.4943157404
Epoch 30, Objective value: 102458.11355095192
Epoch 40, Objective value: -0.39231006311863353
Epoch 50, Objective value: -0.3908168257566796
Epoch 60, Objective value: -0.3836022138577494
Epoch 70, Objective value: -0.38083707640212644
Epoch 80, Objective value: -0.3798465379230715
Epoch 90, Objective value: -0.3794986588509864
Epoch 100, Objective value: -0.3793718050709327
Epoch 0, Objective value: -0.45214920260364233
Epoch 10, Objective value: -0.6959398059921384
Epoch 20, Objective value: -0.6527909554536842
Epoch 30, Objective value: -0.6341584480967435
Epoch 40, Objective value: -0.626911905469902
Epoch 50, Objective value: -0.6240885926594191
Epoch 60, Objective value: -0.623142819549467
Epoch 70, Objective value: -0.6228323692447185
Epoch 80, Objective value: -0.6227699710651788
Epoch 90, Objective value: -0.6227978812773187
Epoch 100, Objective

In [130]:
print('Net Position: ',np.sum(w))
print('Total Position: ',np.sum(np.abs(w)))
print('Beta: ',np.sum(w*beta))
print('Vol: ',100*np.sqrt(w.T@cov@w))

Net Position:  -0.17725991173601208
Total Position:  4.1195293248077265
Beta:  -0.08732596328022876
Vol:  7.97958233543706


In [131]:
#Only for first trade
w_0=w.copy()
w_0[:]=0

In [132]:
(opt.pnl.calc_pnl(100e6,date_new,w_0,w,return_type='all')[0]/100e6)-1

0.024839194649705076

In [133]:
opt.pnl.calc_pnl(100e6,date_new,w_0,w,return_type='all')

(102483919.4649705,
 FOX    -0.000443
 NOW     0.007546
 AMCR    0.010723
 JPM    -0.003563
 KO      0.013953
           ...   
 NWSA   -0.001785
 CCI    -0.016686
 APTV   -0.000541
 LVS     0.001438
 MOH     0.017365
 Length: 497, dtype: float64,
 205976.46624038625,
 9062.964514576995,
 38671.103128893665,
 11719.746057136472,
 2648568.046539446)